In [ ]:
"""
I think 3rd ch may work.
https://www.kaggle.com/c/seti-breakthrough-listen/discussion/245950
thnaks!! please check above discussion!!

code_factory is this [https://github.com/abebe9849/code_factory]

I will update& clean below code if I have time. sorry.

[0,2,4][1,3,5]
3ch(NMF)
add old data (pos only)
"""

In [ ]:


import hydra
from omegaconf import DictConfig, OmegaConf
import sys,gc,os,random,time,math
import matplotlib.pyplot as plt
from contextlib import contextmanager
from pathlib import Path
from collections import defaultdict, Counter
from  torch.cuda.amp import autocast, GradScaler 
import timm
import ttach as tta
from PIL import Image
import numpy as np
import pandas as pd
import scipy as sp
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from functools import partial
from tqdm import tqdm
from sklearn.metrics import precision_score,recall_score,f1_score,log_loss
from  sklearn.metrics import accuracy_score as acc
import torch
import torch.nn as nn
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau,CosineAnnealingWarmRestarts
from torch.utils.data import DataLoader, Dataset
from albumentations import Compose, Normalize, HorizontalFlip, VerticalFlip,RandomGamma, RandomRotate90,GaussNoise,Cutout,RandomBrightnessContrast,RandomContrast,Resize
from albumentations.pytorch import ToTensorV2

from code_factory.pooling import GeM,AdaptiveConcatPool2d
from code_factory.fmix import *
from code_factory.loss_func import SmoothCrossEntropy,MyCrossEntropyLoss,FocalLoss_CE,FocalCosineLoss
#from code_factory.visualisation import *
from code_factory.specaug import *

import logging
#from mylib.
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

import math
import torch
from torch.optim.optimizer import Optimizer

"""
#of cource,torchlibrosa is fast(6８0s/epoch→6４0s)

def spec_augment(spec: np.ndarray,
                 num_mask=2,
                 freq_masking=0.15,
                 time_masking=0.20,
                 value=0):
    spec = spec.copy()
    num_mask = random.randint(1, num_mask)
    for i in range(num_mask):
        all_freqs_num, all_frames_num  = spec.shape
        freq_percentage = random.uniform(0.0, freq_masking)

        num_freqs_to_mask = int(freq_percentage * all_freqs_num)
        f0 = np.random.uniform(low=0.0, high=all_freqs_num - num_freqs_to_mask)
        f0 = int(f0)
        spec[f0:f0 + num_freqs_to_mask, :] = value

        time_percentage = random.uniform(0.0, time_masking)

        num_frames_to_mask = int(time_percentage * all_frames_num)
        t0 = np.random.uniform(low=0.0, high=all_frames_num - num_frames_to_mask)
        t0 = int(t0)
        spec[:, t0:t0 + num_frames_to_mask] = value
    return spec

class SpecAug(ImageOnlyTransform):

    def __init__(self, num_mask=2,freq_masking=0.15,time_masking=0.20, value=0, always_apply=False, p=0.5):
        super().__init__(always_apply, p)
        self.value = value
        self.num_mask = num_mask
        self.freq_masking = freq_masking
        self.time_masking = time_masking

    def apply(self, image, **params):
        image = spec_augment(image,
                 num_mask=self.num_mask,
                 freq_masking=self.freq_masking,
                 time_masking=self.time_masking,
                 value=self.value)
        return image

def spec_augment_m(spec: np.ndarray,
                 num_mask=2,
                 freq_masking=0.15,
                 time_masking=0.20,
                 value=0):
    spec = spec.copy()
    num_mask = random.randint(1, num_mask)
    for i in range(num_mask):
        n_channel, all_freqs_num, all_frames_num  = spec.shape # unpack channel
        freq_percentage = random.uniform(0.0, freq_masking)
        num_freqs_to_mask = int(freq_percentage * all_freqs_num)
        f0 = np.random.uniform(low=0.0, high=all_freqs_num - num_freqs_to_mask)
        f0 = int(f0)
        spec[:, f0:f0 + num_freqs_to_mask, :] = value # augment all channels
        time_percentage = random.uniform(0.0, time_masking) 
        num_frames_to_mask = int(time_percentage * all_frames_num)
        t0 = np.random.uniform(low=0.0, high=all_frames_num - num_frames_to_mask)
        t0 = int(t0)
        spec[:, :, t0:t0 + num_frames_to_mask] = value # augment all channels
    return spec

class SpecAug_m(ImageOnlyTransform):

    def __init__(self, num_mask=2,freq_masking=0.15,time_masking=0.20, value=0, always_apply=False, p=0.5):
        super().__init__(always_apply, p)
        self.value = value
        self.num_mask = num_mask
        self.freq_masking = freq_masking
        self.time_masking = time_masking

    def apply(self, image, **params):
        image = spec_augment_m(image,
                 num_mask=self.num_mask,
                 freq_masking=self.freq_masking,
                 time_masking=self.time_masking,
                 value=self.value)
        return image






"""



from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
def show_cleaned_image_individually(image,plus=50):
    image_on = None
    image_off = None
    clean_image = None
    for i in range(0,6,2):
        temp_on = image[i]
        temp_off = image[i+1]
        temp_on = temp_on + plus
        temp_off = temp_off + plus
        
        model = NMF(init = 'random',
                    n_components = 2,
                    solver = 'mu',
                    alpha = 0.01,
                    random_state = 0,
                   )
        
        W_on = model.fit_transform(temp_on)
        H_on = model.components_

        W_off = model.fit_transform(temp_off)
        H_off = model.components_
        
        temp_clean = normalize(temp_on - np.matmul(W_on, H_off))
        
        if image_off is None:
            image_off = image[i+1]
        else:
            image_off = np.concatenate((image_off, image[i+1]))

        if image_on is None:
            image_on = image[i]
        else:
            image_on = np.concatenate((image_on, image[i]))
            
        if clean_image is None:
            clean_image = temp_clean
        else:
            clean_image = np.concatenate((clean_image, temp_clean))
    return clean_image

def preprocess_image_plus(img,ch_3rd):
    """
    024 as one channel, 135 as the other channel
    """
    information_channels = np.concatenate([img[0], img[2], img[4]])
    helper_channels = np.concatenate([img[1], img[3], img[5]])
    #print(information_channels.shape,c.shape)
    return np.array([ch_3rd,information_channels, helper_channels])


class TrainDataset(Dataset):
    def __init__(self, df,CFG,train=True,transform1=None, transform2=None):
        self.df = df
        self.transform = transform1
        self.transform_ = transform2
        self.CFG = CFG
        self.train = train

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.df['file_path'].values[idx]
        image = np.load(file_path).astype(np.float32)
        p = file_path.split("/")[-1]
        domain = self.df['from_old'].values[idx]
        ## ch_3rd is https://www.kaggle.com/c/seti-breakthrough-listen/discussion/245950
        if domain==0:
            ch_3rd = np.load(f"/home/abebe9849/ET/new_src/clean/new_50/train/{p}")
        else:
            ch_3rd = np.load(f"/data/RSNA/ET/new_data_all/clean_wl/50/old/{p}")
        image = preprocess_image_plus(image,ch_3rd)
        image = image.transpose(2,1,0)
        #if self.train:
        image = self.transform(image=image)['image']
        #image = cv2.resize(image,(768,768))
        image = torch.from_numpy(image.transpose(2,0,1)).float()
        #print(image.shape)

        label_ = self.df["target"].values[idx]
        label = torch.tensor(label_)
        return image, label,torch.tensor(idx)
def sharpen(pred, T):
    pred = pred**T
    pred = pred / np.sum(pred)
    return pred

class semi_test_Dataset(Dataset):
    def __init__(self, df,CFG,train=True,transform1=None, transform2=None):
        self.df = df
        self.transform = transform1
        self.transform_ = transform2
        self.CFG = CFG
        self.train = train

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.df['file_path'].values[idx]

        image = np.load(file_path).astype(np.float32)[[0, 2, 4]]
        image = np.vstack(image).transpose((1, 0))
        if self.train:
            image = self.transform(image=image)['image']
        #image = cv2.resize(image,(self.CFG.preprocess.size,self.CFG.preprocess.size))
        image = torch.from_numpy(image[None,:,:]).float()
        #print(image.shape)

        label_ = self.df["target"].values[idx]
        pred = np.array([1-label_,label_])
        pred =sharpen(pred, 2)
        label = torch.tensor(pred).float()
        return image, label

SEQ_POOLING = {
    'gem': GeM(dim=2),
    'concat': AdaptiveConcatPool2d(),
    'avg': nn.AdaptiveAvgPool2d(1),
    'max': nn.AdaptiveMaxPool2d(1)
}

class Model(nn.Module):
    def __init__(self,CFG, num_classes=2, base_model='tf_efficientnet_b0_ns',pool="avg",pretrain=True):
        super(Model, self).__init__()
        self.base_model = base_model #"str"
        self.CFG = CFG
        self.model = timm.create_model(self.base_model, pretrained=pretrain, num_classes=2,in_chans=3)
        #self.model.conv_stem = nn.Conv2d(2, 32, kernel_size=3, padding=1, stride=1, bias=False)
        nc = self.model.num_features
        if pool in ('avg','concat','gem','max'):
            self.avgpool = SEQ_POOLING[pool]
            if pool == "concat":
                nc *= 2
        self.last_linear = nn.Linear(nc,num_classes)
    def forward(self, input1):
        x = self.model.forward_features(input1)
        if "swin" in self.CFG.model.name:
            feature = x
        else:
            feature = self.avgpool(x).view(input1.size()[0], -1)
        y = self.last_linear(feature)
        return y,feature


from  sklearn.metrics import roc_auc_score,accuracy_score
def AUC(y_true,y_pred,onehot=False):
    if y_pred.shape[1]==2:
        auc = roc_auc_score(y_true,y_pred[:,1])
    else:
        if onehot:
            auc=0
            n_col = y_pred.shape[1]
            print(n_col,y_true[:,i].shape)
            for i in range(n_col):
                auc += roc_auc_score(y_true[:,i],y_pred[:,i])/n_col

        else:
            auc = roc_auc_score(y_true,y_pred,multi_class = "ovr")
    return auc



def train_fn(CFG,fold,folds,test):

    torch.cuda.set_device(CFG.general.device)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"### fold: {fold} ###")
    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index
    train_dataset = TrainDataset(folds.loc[trn_idx].reset_index(drop=True),train=True, 
                                 transform1=get_transforms1(data='train',CFG=CFG),transform2=to_tensor(),CFG=CFG)#
    valid_dataset = TrainDataset(folds.loc[val_idx].reset_index(drop=True),train=False,
                                 transform1=get_transforms1(data='valid',CFG=CFG),transform2=to_tensor(),CFG=CFG)#

    train_loader = DataLoader(train_dataset, batch_size=CFG.train.batch_size, shuffle=True, num_workers=4)
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.train.batch_size, shuffle=False, num_workers=4)
    #if 
    test_dataset = semi_test_Dataset(test,train=True,transform1=get_transforms1(data='train',CFG=CFG),transform2=to_tensor(),CFG=CFG)#
    test_loader = DataLoader(test_dataset, batch_size=CFG.train.batch_size, shuffle=True, num_workers=4)


    ###  model select ============
    if CFG.model.type=="cnn":
        model = Model(num_classes=CFG.model.n_classes,base_model=CFG.model.name,pool=CFG.model.pooling,CFG=CFG)
    elif CFG.model.type=="vit":
        model = Model_vit(num_classes=CFG.model.n_classes,base_model=CFG.model.name,CFG=CFG)
    model.to(device)
    # ============

    ###  optim select ============
    if CFG.train.optim=="adam":
        optimizer = Adam(model.parameters(), lr=CFG.train.lr, amsgrad=False)
    elif CFG.train.optim=="radam":
        optimizer = RAdam(model.parameters(), lr=CFG.train.lr)
    elif CFG.train.optim=="adam_w":
        optimizer = AdamW(model.parameters(), lr=CFG.train.lr)
    # ============

    ###  scheduler select ============
    if CFG.train.scheduler.name=="cosine":
        scheduler = CosineAnnealingLR(optimizer, T_max=CFG.train.epochs, eta_min=CFG.train.scheduler.min_lr)
    elif CFG.train.scheduler.name=="cosine_warm":
        scheduler = CosineAnnealingWarmRestarts(optimizer,T_0=CFG.train.scheduler.t_0, T_mult=1, eta_min=CFG.train.scheduler.min_lr, last_epoch=-1)
    elif CFG.train.scheduler.name=="reduce":
        scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2, verbose=True, eps=1e-6)
    elif CFG.train.scheduler.name=="one":
        scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, total_steps=CFG.train.epochs)
    # ============
    
    ###  loss select ============
    if CFG.augmentation.mix_p>0:
        criterion = SmoothCrossEntropy(smoothing =CFG.loss.smooth_a,one_hotted=True)
    elif CFG.loss.name=="ce" and CFG.loss.weights ==False:
        log.info(f"~~~~class smooth_a{CFG.loss.smooth_a}~~~~~")
        criterion = SmoothCrossEntropy(smoothing =CFG.loss.smooth_a)
    elif CFG.loss.name=="ce" and CFG.loss.weights!=False:
        log.info(f"~~~~class weight{CFG.loss.weights}~~~~~")
        weights = torch.Tensor(CFG.loss.weights).to(device).float()
        #criterion = MyCrossEntropyLoss(weight=weights)
        criterion = nn.CrossEntropyLoss(weight=weights)
    elif CFG.loss.name=="focal" and CFG.loss.weights!=None:
        criterion = FocalLoss_CE(gamma=CFG.loss.focal_gamma)
    elif CFG.loss.name=="focal_cosine":
        criterion = FocalCosineLoss(gamma=CFG.loss.focal_gamma)
    # ============
    print(criterion)
    mse_loss = nn.MSELoss()
    
    softmax = nn.Softmax(dim = 1)
    scaler = torch.cuda.amp.GradScaler()
    best_score = 0
    best_loss = np.inf
    best_preds = None
    
    
    for epoch in range(CFG.train.epochs):
        start_time = time.time()
        model.train()
        avg_loss = 0.

        tk0 = tqdm(enumerate(train_loader), total=len(train_loader))

        for i, (images, labels,indexes) in tk0:
            optimizer.zero_grad()
            images = images.to(device)
            #print(images.shape)
            labels = labels.to(device)
            onehot_label = torch.eye(CFG.model.n_classes)[labels].to(device)

            if CFG.semi_SL.type=="soft":
                input_test, target_test = iter(test_loader).__next__()
                input_test = input_test.to(device)
                target_test = target_test.to(device)


            ### mix系のaugumentation=========
            rand = np.random.rand()
            if epoch+1 >=CFG.train.without_hesitate:
                rand=0

            if CFG.augmentation.mix_p>rand and CFG.augmentation.do_mixup:
                images, y_a, y_b, lam = mixup_data(images, onehot_label,alpha=CFG.augmentation.mix_alpha)
            elif CFG.augmentation.mix_p>rand and CFG.augmentation.do_cutmix:
                images, y_a, y_b, lam = cutmix_data(images, onehot_label,alpha=CFG.augmentation.mix_alpha)
            elif CFG.augmentation.mix_p>rand and CFG.augmentation.do_resizemix:
                images, y_a, y_b, lam = resizemix_data(images, onehot_label,alpha=CFG.augmentation.mix_alpha)
            elif CFG.augmentation.mix_p>rand and CFG.augmentation.do_fmix:
                images, y_a, y_b, lam = fmix_data(images, onehot_label,alpha=CFG.augmentation.mix_alpha)
            ### mix系のaugumentation おわり=========

            if CFG.train.amp:
                with autocast():
                    y_preds,_ = model(images.float())
                    if CFG.semi_SL.type=="soft":
                        y_preds_test,_ = model(input_test.float())
                        loss_mse = mse_loss(y_preds_test,target_test)
                    if CFG.augmentation.mix_p>rand:
                        loss_ = mixup_criterion(criterion, y_preds, y_a, y_b, lam)
                    elif CFG.augmentation.mix_p<=rand and CFG.augmentation.mix_p>0:
                        loss_ = criterion(y_preds,onehot_label) 
                    else:
                        loss_ = criterion(y_preds, labels)

                    if CFG.semi_SL.type=="soft":
                        #print(loss_.dtype,loss_mse.dtype,y_preds_test.dtype,target_test.dtype)
                        loss = loss_+loss_mse*0.05 #
                    else:
                        loss=loss_

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
        if CFG.train.scheduler.name!="none":
            scheduler.step()


            avg_loss += loss.item() / len(train_loader)
        model.eval()
        avg_val_loss = 0.
        preds = []
        valid_labels = []
        tk1 = tqdm(enumerate(valid_loader), total=len(valid_loader))

        for i, (images, labels,indexes) in tk1:
            images = images.to(device)
            labels = labels.to(device)
            onehot_label = torch.eye(CFG.model.n_classes)[labels].to(device)
            if CFG.loss.name=="ce" and CFG.loss.weights!=None:
                labels = labels.long()
            with torch.no_grad():
                with autocast():
                    y_preds,_ = model(images.float())
                    if CFG.augmentation.mix_p>0:
                        loss = criterion(y_preds,onehot_label)
                    else:
                        loss = criterion(y_preds,labels)
            valid_labels.append(labels.to('cpu').numpy())
            softmax = nn.Softmax(dim = 1)
            y_preds = softmax(y_preds)
            preds.append(y_preds.to('cpu').numpy())
            avg_val_loss += loss.item() / len(valid_loader)
        preds = np.concatenate(preds)
        valid_labels = np.concatenate(valid_labels)

        print(preds.shape,valid_labels.shape)


        #else:
        auc_score = AUC(valid_labels,preds)

        elapsed = time.time() - start_time
        log.info(f'  Epoch {epoch+1} - avg_train_loss: {avg_loss:.6f}  avg_val_loss: {avg_val_loss:.6f}  time: {elapsed:.0f}s')
        log.info(f'  Epoch {epoch+1} - AUC : {auc_score:.6f}')
        if auc_score>best_score:#pr_auc best
            best_score = auc_score
            best_preds = preds
            log.info(f'  Epoch {epoch+1} - Save Best AUC: {auc_score:.4f}')
            torch.save(model.state_dict(), f'fold{fold}_{CFG.general.exp_num}_baseline.pth')
    return best_preds, valid_labels

##only shift work

def get_transforms1(*, data,CFG):
    if data == 'train':
        return Compose([
            HorizontalFlip(p=CFG.augmentation.hflip_p),
            VerticalFlip(p=CFG.augmentation.vflip_p),
            albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.0, rotate_limit=0, p=0.75),
            RandomContrast(p=CFG.augmentation.contrast_p),
            #GaussNoise(p=0.5),
            RandomRotate90(p=CFG.augmentation.rotate_90_p),
            #RandomGamma(p=0.5),
            RandomBrightnessContrast(p=CFG.augmentation.bright_contrast_p),
            #GaussianBlur(p=0.5),
            albumentations.Sharpen(alpha=(0.01, 0.1), lightness=(0.01, 0.1), p=0.5),
            GridMask(num_grid=CFG.augmentation.grdimask_n, p=CFG.augmentation.grdimask_p),
            Cutout(p=CFG.augmentation.cutout_p),
            SpecAug_m(num_mask=CFG.augmentation.specAug.num_mask,freq_masking=CFG.augmentation.specAug.freq_masking,
            time_masking=CFG.augmentation.specAug.time_masking,p=CFG.augmentation.specAug.p),
            Resize(CFG.preprocess.size,CFG.preprocess.size),
            #Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
        ])
    elif data == 'valid':
        return Compose([
            Resize(CFG.preprocess.size,CFG.preprocess.size),
            #Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225],
            ])

class TestDataset(Dataset):
    def __init__(self, df,CFG,transform1=None, transform2=None):
        self.df = df
        self.transform = transform1
        self.transform_ = transform2
        self.CFG = CFG

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.df['file_path'].values[idx]
        image = np.load(file_path).astype(np.float32)[[0, 2, 4]]
        image = np.vstack(image).transpose((1, 0))
        #image = self.transform(image=image)['image']
        #image = cv2.resize(image,(self.CFG.preprocess.size,self.CFG.preprocess.size))
        image = torch.from_numpy(image[None,:,:]).float()
        return image

def inference(model, test_loader, device,CFG):
    model.to(device) 
    model.eval()
    probs = []
    features = []
    scaler = torch.cuda.amp.GradScaler()

    for i, images in tqdm(enumerate(test_loader), total=len(test_loader)):
        images = images.to(device)
        with torch.no_grad():
            y_preds,feature = model(images)
            softmax = nn.Softmax(dim = 1)
            y_preds = softmax(y_preds)  
        probs.append(y_preds.to('cpu').numpy())
        features.append(feature.to('cpu').numpy())

    probs = np.concatenate(probs)
    features = np.concatenate(features)
    return probs,features

def inference_tta(model, test_loader, device,CFG):
    transforms = tta.Compose([
        tta.HorizontalFlip(),
        tta.VerticalFlip(),
        #tta.Rotate90(angles=[0, 180]),
        #tta.Multiply(factors=[0.9, 1, 1.1]),        
    ])
    model.to(device) 
    model.eval()
    probs = []

    for i, images in tqdm(enumerate(test_loader), total=len(test_loader)):
        images = images.to(device)
        with torch.no_grad():
            labels = []
            for transformer in transforms:
                augmented_image = transformer.augment_image(images)
                y_preds,_ = model(augmented_image)
                y_preds = transformer.deaugment_label(y_preds)
                softmax = nn.Softmax(dim = 1)
                y_preds = softmax(y_preds)
                labels.append(y_preds)
            labels = torch.stack(labels)
            labels = torch.mean(labels,axis=0)
        probs.append(labels.to('cpu').numpy())

    probs = np.concatenate(probs)
    return probs



def submit(test,CFG):
        print('run inference')
        torch.cuda.set_device(CFG.general.device)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        test_dataset = TestDataset(test,transform1=get_transforms1(data='valid',CFG=CFG),transform2=to_tensor(),CFG=CFG)
        test_loader = DataLoader(test_dataset, batch_size=CFG.train.batch_size, shuffle=False)
        probs = []
        features = []
        for fold in range(5):
            weights_path = f'fold{fold}_{CFG.general.exp_num}_baseline.pth'
            if CFG.model.type=="cnn":
                model = Model(num_classes=CFG.model.n_classes,base_model=CFG.model.name,pool=CFG.model.pooling,CFG=CFG)
            elif CFG.model.type=="vit":
                model = Model_vit(num_classes=CFG.model.n_classes,base_model=CFG.model.name,CFG=CFG)
            state_dict = torch.load(weights_path,map_location=device)
            model.load_state_dict(state_dict)
            if CFG.tta.do:
                _probs = inference_tta(model, test_loader, device,CFG)
                _features = 0
            else:
                _probs,_features = inference(model, test_loader, device,CFG)
            probs.append(_probs)
            features.append(_features)
        probs = np.mean(probs, axis=0)
        features = np.mean(features, axis=0)
        return probs,features

def get_test_file_path(image_id):
    return "/home/abebe9849/ET/data/test/{}/{}.npy".format(image_id[0], image_id)

log = logging.getLogger(__name__)
@hydra.main(config_path="/home/abebe9849/ET/config", config_name="config3")
def main(CFG : DictConfig) -> None:
    #CFG = OmegaConf.to_yaml(cfg)

    seed_torch(seed=42)
    DIR = "/home/abebe9849/ET/new_data"
    log.info(f"===============exp_num{CFG.general.exp_num}============")
    folds = pd.read_csv(f"{DIR}/folds.csv")

    folds["from_old"]=0
    pos = pd.read_csv("/home/abebe9849/ET/new_data/old_cat.csv")
    pos["from_old"]=1
    pos["fold"]=-1
    pos = pos[pos["target"]==1].reset_index(drop=True)
    folds = pd.concat([folds,pos]).reset_index(drop=True)
    test = pd.read_csv(f"{DIR}/test.csv")
    sub = pd.read_csv(f"{DIR}/sample_submission.csv")

    if CFG.semi_SL.psuedo_label!=-1 and CFG.semi_SL.type=="pl":
        inner_test = pd.read_csv(f"{CFG.semi_SL.psuedo_label}/submission.csv")
        inner_test['file_path'] = inner_test['id'].apply(get_test_file_path)

        test2p0 = inner_test[inner_test['target']<CFG.semi_SL.psuedo_th]
        test2p0["target"] = 0
        test2p0["fold"] = -1
        test2p1 = inner_test[inner_test['target']>CFG.semi_SL.psuedo_th_1]
        test2p1["target"] = 1
        test2p0["fold"] = -1



        if len(test2p0)+len(test2p1)>1:
            folds = pd.concat([folds,test2p1,test2p0]).reset_index(drop=True)
        log.info(f"==how many add for PL {len(test2p0)},{len(test2p1)}===")

    if CFG.semi_SL.psuedo_label!=-1 and CFG.semi_SL.type=="soft":
        pre_test = pd.read_csv(f"{CFG.semi_SL.psuedo_label}/submission.csv")
        pre_test['file_path'] = pre_test['id'].apply(get_test_file_path)


    preds = []
    valid_labels = []
    for fold in range(5):
        if CFG.semi_SL.type=="soft":
            _preds, _valid_labels = train_fn(CFG,fold,folds,pre_test)
        else:
            _preds, _valid_labels = train_fn(CFG,fold,folds,test)
        preds.append(_preds)
        valid_labels.append(_valid_labels)
    preds = np.concatenate(preds)
    valid_labels = np.concatenate(valid_labels)

    auc_score = AUC(valid_labels,preds)

    log.info(f'  =====AUC(CV)====== {auc_score}')


    
    if CFG.semi_SL.type=="none":
        folds["pred"] = preds[:,1]
        folds.to_csv("predict.csv",index = False)
    

    

    fpr, tpr, thresholds = metrics.roc_curve(valid_labels, preds[:,1])
    score = metrics.roc_auc_score(valid_labels,preds[:,1])
    plt.plot(fpr, tpr, label=f'AUC={score}')
    plt.legend()
    plt.title(f'ROC curve_train exp{CFG.general.exp_num}')
    plt.xlabel('False Positive Rate(1-Specificity)')
    plt.ylabel('True Positive Rate(Recall)')
    plt.grid(True)
    plt.show()
    plt.savefig("TRAIN_{0}_.png".format(CFG.general.exp_num))


    

    pred,features = submit(test,CFG)

    sub["target"] = pred[:,1]
    sub.to_csv("submission.csv",index = False)

if __name__ == "__main__":
    main()

I used this config.public LB may 0.832,cv=0.8890965123456791
it takes 57min/epoch by q rtx 8000.

In [ ]:
"""
general:
  debug: false
  exp_num: n032_last
  device: 0
  task:
    name: clf
loss:
  name: ce
  weights: false
  smooth_a: 0
  focal_gamma: 0
preprocess:
  size: 768
augmentation:
  grdimask_p: 0
  grdimask_n: 3
  hflip_p: 0.5
  vflip_p: 0.5
  cutout_p: 0.5
  contrast_p: 0
  bright_contrast_p: 0
  rotate_90_p: 0
  bright_p: 0
  do_mixup: true
  do_fmix: false
  do_cutmix: false
  do_snapmix: false
  do_resizemix: false
  mix_p: 0.5
  mix_alpha: 1
  specAug:
    p: 0.5
    num_mask: 2
    freq_masking: 0.15
    time_masking: 0.1
tta:
  do: false
model:
  name: tf_efficientnet_b5_ns
  type: cnn
  pooling: gem
  n_classes: 2
  features_num: 1280
semi_SL:
  type: none
  psuedo_label: none
  psuedo_th: 0.003
  psuedo_th_1: 0.88
train:
  amp: true
  amp_inf: false
  optim: adam
  lr: 0.001
  epochs: 20
  without_hesitate: 15# stop mixup after several epochs
  batch_size: 24
  scheduler:
    name: cosine
    min_lr: 0.00001
    t_0: 3

"""